# Jupyter notebook sample

In [215]:
from requests import session

s = session()
#s.proxies.update({
#    'http': 'socks5h://localhost:9050',
#    'https': 'socks5h://localhost:9050'
#})

resp = s.get('https://check.torproject.org')
print(resp.text)

<!doctype html>
<html lang="en_US">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>
    
      Sorry. You are not using Tor.
    
</title>
  <link rel="icon" type="image/x-icon" href="/torcheck/img/tor-off.png" />
  <style>
    html { height: 100%; }
    body {
      height: 100%;
      text-align: center;
      font-family: Helvetica, sans-serif;
      margin: 0;
    }
    h1 { font-size: 2em; }
    a { text-decoration: none; }
    a:hover { text-decoration: underline; }
    ul { margin: 0; padding: 0; }
    li {
      text-align: left;
      display: inline;
      list-style-type: none;
      margin: 0;
    }
    .content {
      padding: 4em;
    }
    .foot {
      padding: 2em 4em;
      clear: both;
    }
    .head {
      background-color: #85559f;
      padding: 0.8em 0;
      color: #fff;
    }
    .project {
      margin: 0;
      font-size: 0.8em;
    }
    
  .on { color: green; }
  .off { color: red; }


# deepdarkCTI

In [52]:
import csv
import re
import pandas as pd

deepdarkcti_md = s.get('https://raw.githubusercontent.com/fastfire/deepdarkCTI/refs/heads/main/ransomware_gang.md').text
lines = deepdarkcti_md.splitlines()
lines.pop(1) # Header-Endzeile entfernen
lines = [line.strip('|') for line in lines] # '|' Am Anfang und Ende der Zeile entfernen

deepdarkcti_reader = csv.DictReader(lines, delimiter='|')

deepdarkcti = { 'Group': [], 'Link': [], 'Available': [] }
for row in deepdarkcti_reader:
    parsed_name = re.search(r"\[([^]]*)]\(([^)]*)\)", row['Name'])
    if parsed_name is not None:
        if parsed_name.group(1) in [
            'Ransomfeed',
            'eCrime Services',
            'RANSOM DB',
            'RANSOMWARE GROUP SITES (list)',
            'RANSOMWARE GROUPS MONITORING TOOL'
        ]:
            continue
        deepdarkcti['Group'].append(parsed_name.group(1))
        deepdarkcti['Link'].append(parsed_name.group(2))
        deepdarkcti['Available'].append('ONLINE' in row['Status'])
        
deepdarkcti = pd.DataFrame(deepdarkcti)
deepdarkcti

,Group,Link,Available
0,54bb47h (Victims page),http://54bb47h5qu4k7l4d7v5ix3i6ak6elysn3net4by...,False
1,54bb47h,http://54bb47h.blog,False
2,54bb47h,http://54bb47h5qu4k7l4d7v5ix3i6ak6elysn3net4by...,False
3,0mega (Dark),http://omegalock5zxwbhswbisc42o2q2i54vdulyvtqq...,True
4,0mega (Deep),https://0mega.cc,False
...,...,...,...
405,XING LOCKER,http://xingnewj6m4qytljhfwemngm7r7rogrindbq7wr...,False
406,XINOF,http://wj3b2wtj7u2bzup75tzhnso56bin6bnvsxcbwbf...,False
407,YANLUOWANG,http://jukswsxbh3jsxuddvidrjdvwuohtsy4kxg2axbp...,False
408,ZEON (Victims page),http://zeonrefpbompx6rwdqa5hxgtp2cxgfmoymlli3a...,False


# Reddit

In [58]:
import bs4

reddit2024_soup = bs4.BeautifulSoup(s.get('https://www.reddit.com/r/Malware/comments/1bpcrdw/list_of_ransomware_groups_and_their_pr_pages_2024/').text)
reddit2024_soup = reddit2024_soup.css.select('shreddit-post [slot=text-body] > div > div > p')

reddit2024 = { 'Group': [], 'Link': [], 'Available': [] }
for p in reddit2024_soup:
    if p.find('a') is None:
        continue
    parsed_text = re.search(r"([^]]*) - ([^)]*)", p.text)
    if parsed_text is not None:
        reddit2024['Group'].append(parsed_text.group(1).strip())
        reddit2024['Link'].append(parsed_text.group(2).strip())
        print(f"Checking {parsed_text.group(1).strip()} ({parsed_text.group(2).strip()})")
        try:
            s.get(parsed_text.group(2).strip())
            reddit2024['Available'].append(True)
        except:
            reddit2024['Available'].append(False)
            
reddit2024 = pd.DataFrame(reddit2024)
reddit2024

Checking LockBit 3.0 (http://lockbit3753ekiocyo5epmpy6klmejchjtzddoekjlnt6mu3qh4de2id.onion/)
Checking Inc Ransom (http://incblog7vmuq7rktic73r4ha4j757m3ptym37tyvifzp2roedyyzzxid.onion/)
Checking Qilin (http://kbsqoivihgdmwczmxkbovk7ss2dcynitwhhfu5yw725dboqo5kthfaad.onion/)
Checking Blackbasta (http://stniiomyjliimcgkvdszvgen3eaaoz55hreqqx6o77yvmpwt7gklffqd.onion/)
Checking Play (http://mbrlkbtq5jonaqkurjwmxftytyn2ethqvbxfu4rgjbkkknndqwae6byd.onion/)
Checking Akira (https://akiral2iz6a7qgd3ayp3l6yub7xx2uep76idk3u2kollpj5z3z636bad.onion/)
Checking Cactus (https://cactusbloguuodvqjmnzlwetjlpj6aggc6iocwhuupb47laukux7ckid.onion)
Checking Everest (http://ransomocmou6mnbquqz44ewosbkjk3o5qjsl3orawojexfook2j7esad.onion/)
Checking Ransomhub (http://ransomxifxwc5eteopdobynonjctkxxvap77yqifu2emfbecgbqdw6qd.onion/)
Checking Abyss-data (http://3ev4metjirohtdpshsqlkrqcmxq6zu3d7obrdhglpy5jpbr7whmlfgqd.onion/)
Checking 8base (http://xb6q2aggycmlcrjtbjendcnnwpmmwbosqaugxsqb4nx6cmod3emy7sad.onion/)
Chec

,Group,Link,Available
0,LockBit 3.0,http://lockbit3753ekiocyo5epmpy6klmejchjtzddoe...,True
1,Inc Ransom,http://incblog7vmuq7rktic73r4ha4j757m3ptym37ty...,False
2,Qilin,http://kbsqoivihgdmwczmxkbovk7ss2dcynitwhhfu5y...,False
3,Blackbasta,http://stniiomyjliimcgkvdszvgen3eaaoz55hreqqx6...,True
4,Play,http://mbrlkbtq5jonaqkurjwmxftytyn2ethqvbxfu4r...,True
5,Akira,https://akiral2iz6a7qgd3ayp3l6yub7xx2uep76idk3...,False
6,Cactus,https://cactusbloguuodvqjmnzlwetjlpj6aggc6iocw...,False
7,Everest,http://ransomocmou6mnbquqz44ewosbkjk3o5qjsl3or...,True
8,Ransomhub,http://ransomxifxwc5eteopdobynonjctkxxvap77yqi...,True
9,Abyss-data,http://3ev4metjirohtdpshsqlkrqcmxq6zu3d7obrdhg...,True


# Onion Site

In [60]:
import bs4

darknet_soup = bs4.BeautifulSoup(s.get('http://ransomwr3tsydeii4q43vazm7wofla5ujdajquitomtd47cxjtfgwyyd.onion/').text)
darknet_soup = darknet_soup.css.select_one('.table .table-body')
darknet_soup = darknet_soup.css.select('.table-row')

darknet = { 'Group': [], 'Link': [], 'Available': [] }
for row in darknet_soup:
    name = row.css.select_one('.table-cell')
    links = row.css.select('a')
    for link in links:
        darknet['Group'].append(name.text)
        darknet['Link'].append(link.attrs['href'])
        print(f"Checking {name.text} ({link.attrs['href']})")
        try:
            s.get(link.attrs['href'])
            darknet['Available'].append(True)
        except:
            darknet['Available'].append(False)

darknet = pd.DataFrame(darknet)
darknet

,Group,Link,Available
0,Arvin Club,http://3kp6j22pz3zkv76yutctosa6djpj4yib2icvdqx...,False
1,Babuk,http://nq4zyac4ukl4tykmidbzgdlvaboqeqsemkp4t35...,False
2,Black Basta,https://stniiomyjliimcgkvdszvgen3eaaoz55hreqqx...,False
3,AlphaVM/BlackCat,http://alphvmmm27o3abo3r2mlmjrpdmzle3rykajqc5x...,False
4,BlackByte,http://ce6roic2ykdjunyzazsxmjpz5wsar4pflpoqznt...,False
5,Bl4ckt0r,http://bl4cktorpms2gybrcyt52aakcxt6yn37byb65ua...,False
6,CL0P,http://santat7kpllt6iyvqbr7q4amdv6dzrh6paatvyr...,True
7,CONTI,http://continewsnv5otx5kaoje7krkto2qbu3gtqef22...,False
8,CRYP70N1C0D3,http://7k4yyskpz3rxq5nyokf6ztbpywzbjtdfanweup3...,False
9,Cuba,http://cuba4ikm4jakjgmkezytyawtdgr2xymvy6nvzgw...,False


# Ransom-db

In [141]:
import bs4

ransomdb = { 'Group': [], 'Available': [] }
with open("data/ransomdb.html") as f:
    ransomdb_soup = bs4.BeautifulSoup(f.read())
    ransomdb_soup = ransomdb_soup.find('tbody')
    ransomdb_soup = ransomdb_soup.css.select('tr:not(:first-child)')

    for row in ransomdb_soup:
        badge = row.css.select_one('.badge')
        if badge is None:
            continue
        name = row.css.select_one('td:nth-child(2)')
        #ransomdb['Group'].append(name.text)
        #ransomdb['Available'].append('Online' in badge.text)

ransomdb = pd.DataFrame(ransomdb)
ransomdb

,Group,Available


# Ransomwatch

In [80]:
import csv

ransomwatch_md = s.get('https://ransomwatch.telemetry.ltd/INDEX.md').text
lines = ransomwatch_md.splitlines()[2:-1]
lines.pop(1) # Header-Endzeile entfernen
lines = [line.strip('|') for line in lines] # '|' Am Anfang und Ende der Zeile entfernen

ransomwatch_reader = csv.DictReader(lines, delimiter='|')

ransomwatch = { 'Group': [], 'Available': [], 'Link': [] }
for row in ransomwatch_reader:
    if len(row[' group '].strip()) == 0:
        continue
    parsed_name = re.search(r"\[([^]]*)]\(([^)]*)\)", row[' group '])
    ransomwatch['Group'].append(parsed_name.group(1))
    ransomwatch['Link'].append(row[' location '])
    if row[' status '] is not None:
        ransomwatch['Available'].append('🟢' in row[' status '])
    else:
        try:
            s.get(row[' location '])
            ransomwatch['Available'].append(True)
        except:
            ransomwatch['Available'].append(False)
        
    
ransomwatch = pd.DataFrame(ransomwatch)
ransomwatch

,Group,Available,Link
0,marketo,False,marketojbwagqnwx.onion
1,marketo,False,g5sbltooh2okkcb2.onion
2,marketo,False,fvki3hj7uxuirxpeop6chgqoczanmebutznt2mkzy6wao...
3,marketo,False,jvdamsif53dqjycuozlaye2s47p7xij4x6hzwzwhzrqmv...
4,synack,False,xqkz2rmrqkeqf6sjbrb47jfwnqxcd4o2zvaxxzrpbh2pi...
...,...,...,...
446,lynx,False,lynxbllrfr5262yvbgtqoyq76s7mpztcqkv6tjjxgpilp...
447,helldown,False,onyxcgfg4pjevvp5h34zvhaj45kbft3dg5r33j5vu3nyp...
448,orca,True,orca66hwnpciepupe5626k2ib6dds6zizjwuuashz67us...
449,nitrogen,True,nitrogenczslprh3xyw6lh5xyjvmsz7ciljoqxxknd7uy...


# Ransomfind

In [68]:
import csv

ransomfind_md = s.get('https://ransomfind.io/INDEX.md').text
lines = ransomfind_md.splitlines()[2:-1]
lines.pop(1) # Header-Endzeile entfernen
lines = [line.strip('|') for line in lines] # '|' Am Anfang und Ende der Zeile entfernen

ransomfind_reader = csv.DictReader(lines, delimiter='|')

ransomfind = { 'Group': [], 'Available': [], 'Link': [] }
for row in ransomfind_reader:
    parsed_name = re.search(r"\[([^]]*)]\(([^)]*)\)", row[' group '])
    ransomfind['Group'].append(parsed_name.group(1))
    ransomfind['Link'].append(row[' location '].strip())
    ransomfind['Available'].append('🟢' in row[' status '])

ransomfind = pd.DataFrame(ransomfind)
ransomfind

,Group,Available,Link
0,marketo,False,marketojbwagqnwx.onion
1,marketo,False,g5sbltooh2okkcb2.onion
2,marketo,False,fvki3hj7uxuirxpeop6chgqoczanmebutznt2mkzy6waov...
3,marketo,False,jvdamsif53dqjycuozlaye2s47p7xij4x6hzwzwhzrqmv3...
4,synack,False,xqkz2rmrqkeqf6sjbrb47jfwnqxcd4o2zvaxxzrpbh2pik...
...,...,...,...
234,mallox,True,wtyafjyhwqrgo4a45wdvvwhen3cx4euie73qvlhkhvlrex...
235,projectrelic,False,relic5zqwemjnu4veilml6prgyedj6phs7de3udhicuq53...
236,play,True,k7kg3jqxang3wh7hnmaiokchk7qoebupfgoik6rha6mjpz...
237,play,True,mbrlkbtq5jonaqkurjwmxftytyn2ethqvbxfu4rgjbkkkn...


# ransomlook

In [216]:
import bs4

ransomlook_soup = bs4.BeautifulSoup(s.get('https://www.ransomlook.io/groups').text)

ransomlook = { 'Group': [], 'Available': [], 'Link': [] }
for header in ransomlook_soup.css.select('h3'):
    following = header.find_next('div')
    if following is not None:
        links = following.css.select_one('tbody')
        for link in links.css.select('tr'):
            ransomlook['Group'].append(header.text)
            ransomlook['Link'].append(link.css.select_one('td:nth-child(4)').text)
            ransomlook['Available'].append('⬆️' in link.css.select_one('td:nth-child(2)').text)

ransomlook = pd.DataFrame(ransomlook)
ransomlook


,Group,Available,Link
0,0Mega,True,http://omegalock5zxwbhswbisc42o2q2i54vdulyvtq...
1,0Mega,True,https://0mega.cc/
2,0Mega,False,https://0mega.ws/
3,3Am,True,http://threeamkelxicjsaf2czjyz2lc4q3ngqkxhhle...
4,3Am,True,http://ulkvlj5sirgrbnvb4hvbjo2ex2c2ceqe2j4my5...
...,...,...,...
663,Xinglocker,False,http://xingnewj6m4qytljhfwemngm7r7rogrindbq7w...
664,Xinof,False,http://wj3b2wtj7u2bzup75tzhnso56bin6bnvsxcbwb...
665,Yanluowang,False,http://jukswsxbh3jsxuddvidrjdvwuohtsy4kxg2axb...
666,Zeon,False,http://zeonrefpbompx6rwdqa5hxgtp2cxgfmoymlli3...


# ransomfeed

In [231]:
import bs4

ransomfeed = { 'Group': [], 'Link': [], 'Available': [] }
with open("data/ransomfeed.html") as f:
    ransomfeed_soup = bs4.BeautifulSoup(f.read())
    ransomfeed_soup = ransomfeed_soup.find('tbody')
    ransomfeed_soup = ransomfeed_soup.css.select('tr')

    for row in ransomfeed_soup:
        name = row.css.select_one('a')
        group_soup = bs4.BeautifulSoup(s.get(f"https://ransomfeed.it{name.attrs['href']}").text)
        card = group_soup.css.select_one('.card:nth-child(3)')
        if card is None:
            continue
        for ro in card.css.select('tr:not(:first-child)'):
            ransomfeed['Group'].append(name.text)
            ransomfeed['Link'].append(ro.css.select_one('td:nth-child(1)').text)
            ransomfeed['Available'].append('🟢' in ro.css.select_one('td:nth-child(3)').text)

ransomfeed = pd.DataFrame(ransomfeed)
ransomfeed

,Group,Link,Available
0,0mega,omegalock5zxwbhswbisc42o2q2i54vdulyvtqqbudqous...,True
1,0mega,0mega.cc,True
2,8base,basemmnnqwxevlymli5bs36o5ynti55xojzvn246spahni...,False
3,8base,xb6q2aggycmlcrjtbjendcnnwpmmwbosqaugxsqb4nx6cm...,False
4,8base,92.118.36.204.,True
...,...,...,...
354,werewolves,werewolves.pro,True
355,werewolves,weerwolven.biz,True
356,xinglocker,xingnewj6m4qytljhfwemngm7r7rogrindbq7wrfeepejg...,False
357,yanluowang,jukswsxbh3jsxuddvidrjdvwuohtsy4kxg2axbppiyclom...,False


In [232]:
import bs4

ransomwarelive = {'Group': [], 'Link': [], 'Available': []}
ransomwarelive_soup = bs4.BeautifulSoup(s.get('https://www.ransomware.live/groups/').text)
ransomwarelive_soup = ransomwarelive_soup.find('tbody')
ransomwarelive_soup = ransomwarelive_soup.css.select('tr')

for row in ransomwarelive_soup:
    name = row.css.select_one('a')
    group_soup = bs4.BeautifulSoup(s.get(f"https://www.ransomware.live{name.attrs['href']}").text)
    card = group_soup.css.select_one('tbody')
    if card is None:
        continue
    for ro in card.css.select('tr'):
        ransomwarelive['Group'].append(name.text)
        ransomwarelive['Link'].append(ro.css.select_one('td:nth-child(4)').text)
        ransomwarelive['Available'].append('🟢' in ro.css.select_one('td:nth-child(2)').text)

ransomwarelive = pd.DataFrame(ransomfeed)
ransomwarelive



,Group,Link,Available
0,0mega,omegalock5zxwbhswbisc42o2q2i54vdulyvtqqbudqous...,True
1,0mega,0mega.cc,True
2,8base,basemmnnqwxevlymli5bs36o5ynti55xojzvn246spahni...,False
3,8base,xb6q2aggycmlcrjtbjendcnnwpmmwbosqaugxsqb4nx6cm...,False
4,8base,92.118.36.204.,True
...,...,...,...
354,werewolves,werewolves.pro,True
355,werewolves,weerwolven.biz,True
356,xinglocker,xingnewj6m4qytljhfwemngm7r7rogrindbq7wrfeepejg...,False
357,yanluowang,jukswsxbh3jsxuddvidrjdvwuohtsy4kxg2axbppiyclom...,False


In [233]:
from urllib.parse import urlparse


def normalize_url(url):
    if pd.isna(url):
        return None
    netloc = urlparse(url.strip()).netloc
    if len(netloc.strip()) == 0:
        return url.strip()
    return netloc

def normalize_group(name):
    name = re.search(r"(.*?)( \(.*)?$", name.lower().strip())
    name = name.group(1)
    name = name.replace(' - new', '')
    name = name.replace(' locker', '')
    name = name.replace(' archives', '')
    name = name.replace('-data', '')
    name = name.replace(' security', '')
    name = name.replace(' media', 'media')
    name = name.replace(' vault', 'vault')
    name = name.replace(' vault', 'vault')
    name = name.replace('donut blog', 'donutleaks')
    name = name.replace('donut dls', 'donutleaks')
    name = name.replace('dragon force', 'dragonforce')
    name = name.replace('el dorado/blacklock', 'el dorado')
    name = name.replace('eldorado', 'el dorado')
    name = name.replace(' international', '')
    name = name.replace('kelvinsecurity', 'kelvin')
    name = name.replace('kill 3.0', 'killsec3')
    name = name.replace('killsecurity', 'killsec')
    name = name.replace('lockbit 3.0', 'lockbit3')
    name = name.replace('lockbit3_fs', 'lockbit3')
    name = name.replace('malek team', 'malekteam')
    name = name.replace('money message', 'moneymessage')
    name = name.replace('moses staff', 'mosesstaff')
    name = name.replace('mydata / alpha', 'mydata')
    name = name.replace('ragnar_locker', 'ragnarlocker')
    name = name.replace('ragnar', 'ragnarlocker')
    name = name.replace('ra group', 'ragroup')
    name = name.replace('ransom house', 'ransomhouse')
    name = name.replace('ransom hub', 'ransomhub')
    name = name.replace('ra world', 'raworld')
    name = name.replace('space bears', 'spacebears')
    name = name.replace('team ', '')
    name = name.replace('vanirgroup', 'vanir group')
    name = name.replace('vanir', 'vanir group')
    name = name.replace('werewolves group', 'werewolves')
    return name

deepdarkcti['Link'] = deepdarkcti['Link'].transform(normalize_url)
deepdarkcti['Group'] = deepdarkcti['Group'].transform(normalize_group)
reddit2024['Link'] = reddit2024['Link'].transform(normalize_url)
reddit2024['Group'] = reddit2024['Group'].transform(normalize_group)
darknet['Link'] = darknet['Link'].transform(normalize_url)
darknet['Group'] = darknet['Group'].transform(normalize_group)
ransomwatch['Link'] = ransomwatch['Link'].transform(normalize_url)
ransomwatch['Group'] = ransomwatch['Group'].transform(normalize_group)
#ransomdb['Link'] = ransomdb['Link'].transform(normalize_url)
#ransomdb['Group'] = ransomdb['Group'].transform(normalize_group)
ransomfind['Link'] = ransomfind['Link'].transform(normalize_url)
ransomfind['Group'] = ransomfind['Group'].transform(normalize_group)
ransomlook['Link'] = ransomlook['Link'].transform(normalize_url)
ransomlook['Group'] = ransomlook['Group'].transform(normalize_group)
ransomwarelive['Link'] = ransomwarelive['Link'].transform(normalize_url)
ransomwarelive['Group'] = ransomwarelive['Group'].transform(normalize_group)
ransomfeed['Link'] = ransomfeed['Link'].transform(normalize_url)
ransomfeed['Group'] = ransomfeed['Group'].transform(normalize_group)

ransom_groups = pd.concat([
    deepdarkcti,
    reddit2024,
    darknet,
    ransomdb,
    ransomwatch,
    ransomfind,
    ransomlook,
    ransomwarelive,
    ransomfeed
])

tracking_status = {
    'Group': [],
    'deepdarkcti': [],
    'reddit2024': [],
    'darknet': [],
    'ransomwatch': [],
    'ransomfind': [],
    'ransomlook': [],
    'ransomwarelive': [],
    'ransomfeed': []
}

for group in ransom_groups[ransom_groups['Available'] == 1]['Group'].unique():
    tracking_status['Group'].append(group)
    tracking_status['deepdarkcti'].append((deepdarkcti['Available'] & (deepdarkcti['Group'] == group)).any())
    tracking_status['reddit2024'].append((reddit2024['Available'] & (reddit2024['Group'] == group)).any())
    tracking_status['darknet'].append((darknet['Available'] & (darknet['Group'] == group)).any())
    tracking_status['ransomwatch'].append((ransomwatch['Available'] & (ransomwatch['Group'] == group)).any())
    tracking_status['ransomfind'].append((ransomfind['Available'] & (ransomfind['Group'] == group)).any())
    tracking_status['ransomlook'].append((ransomlook['Available'] & (ransomlook['Group'] == group)).any())
    tracking_status['ransomwarelive'].append((ransomwarelive['Available'] & (ransomwarelive['Group'] == group)).any())
    tracking_status['ransomfeed'].append((ransomfeed['Available'] & (ransomfeed['Group'] == group)).any())

tracking_status = pd.DataFrame(tracking_status)
tracking_status

/tmp/ipykernel_15293/3587825870.py:70: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  ransom_groups = pd.concat([


,Group,deepdarkcti,reddit2024,darknet,ransomwatch,ransomfind,ransomlook,ransomwarelive,ransomfeed
0,0mega,True,False,False,True,True,True,True,True
1,3am,True,True,False,False,False,True,False,False
2,8base,True,False,False,True,False,True,True,True
3,abrahams ax,True,False,False,False,False,False,False,False
4,abyss,True,True,False,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...
143,robinhood,False,False,False,False,False,True,True,True
144,rtm,False,False,False,False,False,True,False,False
145,soleenya,False,False,False,False,False,True,False,False
146,vanir group group group,False,False,False,False,False,True,True,True


In [ ]:
import json
import pandas as pd

keywords = [
    "medical",
    "hospital",
    "clinic",
    "insur",
    "dent",
    "patient",
    "recover",
    "doctor",
    "nurs",
    "health",
    "facilit",
    "care",
    "lab",
    "pediat",
    "rehab",
    "krank",
    "mediz",
]

with open("data/posts.json", "r") as f:
    posts = json.load(f)

df = pd.json_normalize(posts)

filtered = df['post_title'].apply(lambda d: any(keyword in d.lower() for keyword in keywords))

df = df[filtered]

df['discovered'] = pd.to_datetime(df['discovered'])
df['discovered'] = df['discovered'].dt.to_period('M')

In [ ]:
results = { 'Title': [], 'Group': [], 'Date': [] }
for post in posts:
    title = post['post_title'].lower()
    for keyword in keywords:
        if keyword in title:
            results['Title'].append(post['post_title'])
            results['Group'].append(post['group_name'])
            results['Date'].append(post['discovered'])
            break

df = DataFrame(results)

df['Date'] = to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M')
print(len(results['Title']))


data = df['Month'].value_counts().head(100)
data.plot()